# Fonction Python et symbolique

## Résumé

Dans cette page, nous présentons comment définir des fonctions Python et symbolique.

## Références

* User Manual, Functions : http://openturns.github.io/openturns/master/user_manual/functions.html
* Examples, Functional modeling : http://openturns.github.io/openturns/master/examples/functional_modeling/functional_modeling.html

* http://openturns.github.io/openturns/master/user_manual/_generated/openturns.MemoizeFunction.html
* Sur ExprTk : OpenTURNS Users’ Day #11, Friday, the 15 th, June 2018, Denis Barbier, http://trac.openturns.org/blog/OpenTURNS_Users_Day_11

## Exemple

Pour chaque fonction, nous illustrons la connexion avec l’exemple suivant :
* 3 entrées, de loi normale standard, indépendantes
* 2 sorties
* Formule symbolique

La formule symbolique est donnée par :
$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 - X_2 X_3
\end{eqnarray}
$$

Les résultats exacts sont les suivants.

| Variable | Espérance | Ecart-type |
|-|-|-|
| $Y_1$ | 0 | 1.732 |
| $Y_2$ | 0 | 1.415 |


In [1]:
import openturns as ot
X0 = ot.Normal(0.,1.)
X1 = ot.Normal(0.,1.)
X2 = ot.Normal(0.,1.)
inputDistribution = ot.ComposedDistribution((X0,X1,X2))
inputRandomVector = ot.RandomVector(inputDistribution)

ModuleNotFoundError: No module named 'openturns'

## PythonFunction : constructeur

La classe `PythonFunction` permet de créer une fonction OpenTURNS en utilisant une fonction Python créée avec l'opérateur `def`. 

Le constructeur de la classe PythonFunction est

`PythonFunction ( nbInputs , nbOutputs , myPythonFunc )`

où
* `nbInputs` : nombre de variables d’entrées,
* `nbOutputs` : nombre de variables de sorties,
* `myPythonFunc` : une fonction Python.

Le simulateur `mySimulator` a la séquence d'appel `y=mySimulator(x)` où
* `x` : l’entrée du simulateur, un vecteur de taille `nbInputs`,
* `y` : la sortie du simulateur, un vecteur de taille `nbOutputs`.

In [ ]:
def mySimulator(x):
    y0=x[0]+x[1]+x[2]
    y1=x[0]-x[1]*x[2]
    y=[y0,y1]
    return y

## Exemple d'utilisation de la PythonFunction

Dans l'exemple suivant, on estime la moyenne par Monte-Carlo simple sur la base de 10000 expériences.

In [ ]:
myfunction = ot.PythonFunction (3 ,2 , mySimulator )
outputVect =  ot.RandomVector(myfunction, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)
empiricalMean = outputSample.computeMean()
print(empiricalMean)
empiricalSd = outputSample.computeStandardDeviationPerComponent()
print(empiricalSd)

## Quel type pour x, pour y ?

| Type | Entrée X | Sortie Y |
|--|--|--|
|list (Python) | - | ✓ |
|tuple (Python) | - | ✓ |
|array (NumPy) | - | ✓ |
|Point (OpenTURNS) | ✓ | ✓ |


## PythonFunction : objectifs, avantages, inconvénients

Les objectifs de la classe `PythonFunction` sont :

* Simplicité de mise en oeuvre.
* Fonction directement en Python : possibilité d’utiliser tous les modules Python pour réaliser le calcul, ou la connexion.

Avantages :

* Utile si le simulateur est disponible en Python.
* Possibilité de vectorisation avec l’option `func_sample`.

Inconvénients :

* Pas multithread.

## PythonFunction vectorisée : objectifs, avantages, inconvénients

La classe PythonFunction possède une option `func_sample` :

* Idée : améliorer la performance en vectorisant les opérations.
* Principe : évaluer toutes les sorties en fonction de toutes les entrées en un seul appel à la fonction, sans boucle `for`.
* Implémentation : l’entrée et la sortie sont des matrices (et non plus des vecteurs).

Avantages :

* Amélioration de la performance.

Inconvénients :

* Nécessite de vectoriser le calcul.

## Prototype

```
def mySimulator (x):
    [...]
    return y
myfunction=PythonFunction(nbInputs, nbOutputs, func_sample=mySimulator)
```

où
* x : l’entrée du simulateur, un `Sample` de taille `nbExperiments` (`getSize`), de dimension `nbInputs` (`getDimension`),
* y : la sortie du simulateur
  * un `array` : `nbExperiments` lignes et `nbOutputs` colonnes
  * un `Sample` : taille `nbExperiments` et dimension `nbOutputs`

## PythonFunction vectorisée : exemple avec Numpy


In [ ]:
import numpy as np
def mySimulatorVect (x):
    # Conversion Sample > Array Numpy
    x= np.array (x)
    x0=x[: ,0] # Extraction de la colonne 0
    x1=x[: ,1]
    x2=x[: ,2]
    y0=x0+x1+x2
    y1=x0-x1*x2
    # Empilement de deux lignes
    y= np.vstack ((y0 ,y1 ))
    y=y.transpose ()
    return y
myfunctionVect = ot.PythonFunction (3 ,2 , func_sample = mySimulatorVect )

In [ ]:
outputVect =  ot.RandomVector(myfunctionVect, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)

empiricalMean = outputSample.computeMean()
print(empiricalMean)
empiricalSd = outputSample.computeStandardDeviationPerComponent()
print(empiricalSd)

## MemoizeFunction pour gérer l'historique

La classe `MemoizeFunction` définit un mécanisme d’historique des appels à G.

| Méthodes | Fonction |
|-|-|
| `enableHistory()` | active l’historique (défaut : activé) |
| `disableHistory()` | désactive l’historique |
| `isHistoryEnabled()` | vrai si l’historique est actif |
| `clearHistory()` | vide l’historique |
| `getInputHistory()` | un `Sample`, historique des entrées X |
| `getOutputHistory()` | un `Sample`, historique des sorties Y |

In [ ]:
myfunction = ot.PythonFunction (3 ,2 , mySimulator )
myfunction = ot.MemoizeFunction(myfunction)

In [ ]:
outputVariableOfInterest =  ot.RandomVector(myfunction, inputRandomVector)
montecarlosize = 10
outputSample = outputVariableOfInterest.getSample(montecarlosize)

Récupère l'historique en entrée.

In [ ]:
inputs = myfunction.getInputHistory()
inputs

## Fonction symbolique

La classe `SymbolicFunction` peut créer des fonctions symboliques :

* Idée : utiliser une fonction symbolique simple.
* Principe : fournir une chaîne de caractère définissant le calcul.

Avantages :

* Amélioration de la performance.
* Calcul automatique du gradient exact, de la Hessienne exacte

Inconvénients :

* Nécessite une formule mathématique simple, typiquement une seule ligne (mais cette limitation peut être levée avec ExprTk)

## Fonction symbolique : prototype

Prototype :

`
myfunction = SymbolicFunction (liste_des_entrees ,liste_des_formules )
`

où
* `liste_des_entrees` : une liste de `nbInputs` chaînes de caractères, le nom des variables d’entrées
* `liste_des_formules` : une liste de `nbOutputs` chaînes de caractères, les formules de calcul.

In [ ]:
import openturns as ot
X0 = ot.Normal(0.,1.)
X1 = ot.Normal(0.,1.)
X2 = ot.Normal(0.,1.)
inputDistribution = ot.ComposedDistribution((X0,X1,X2))
inputRandomVector = ot.RandomVector(inputDistribution)

In [ ]:
myfunction = ot.SymbolicFunction(("x0","x1","x2"),("x0+x1+x2","x0-x1*x2"))
myfunction = ot.MemoizeFunction(myfunction)

In [ ]:
outputVect =  ot.RandomVector(myfunction, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)
empiricalMean = outputSample.computeMean()
print(empiricalMean)

In [ ]:
outputs = myfunction.getOutputHistory()
outputs[1:10,:]

##  Exercices

### Exercice 1 : une fonction avec 4 entrées

On considère un nouveau modèle, avec une nouvelle variable de sortie
Y3 et une nouvelle variable d’entrée X4 :
$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 - X_2 X_3 \\
Y_3 &=& 2 X_1 + 3 X_2 + 4 X_4
\end{eqnarray}
$$

**Questions**
* Modifier la fonction Python pour simuler le nouveau modèle.
* Ajouter une nouvelle variable X4 de loi normale standard dans le modèle probabiliste.
* Estimer la moyenne de la sortie par Monte-Carlo simple.

### Exercice 2 : gradient d'une fonction Python

OT peut calculer la dérivée approchée d’une fonction Python par différences finies. On peut paramétrer la formule de différence utilisée, ainsi que le pas de différenciation de cette formule. De plus, lorsque la matrice Jacobienne est implémentée dans une fonction Python, on peut transmettre cette fonction à OpenTURNS pour qu'il l'utilise.

**Questions**
* Définir la fonction `myfunction` comme dans le sujet, c'est à dire avec 3 entrées et 2 sorties.
* Utiliser la méthode `gradient` de l’objet `myfunction` pour évaluer le
gradient G'(x) au point d’entrée X = (1, 2, 3).
* Utiliser la méthode `hessian` de l’objet `myfunction` pour évaluer la
matrice Hessienne de G.

* Utiliser les instructions suivantes pour configurer un gradient calculé
par une formule de différences finies décentrée, avec un pas h = 10−2.

```
functionImpl = myfunction.getEvaluation()
h = 1.e -2
myGradient = ot.NonCenteredFiniteDifferenceGradient(h, functionImpl)
myfunction.setGradient ( myGradient )
```

* Evaluer à nouveau le gradient avec la méthode gradient et comparer avec le résultat précédent.
* On peut transmettre à OT une fonction Python qui évalue le gradient. Pour cela on peut utiliser la séquence d'appel :
```
myfunction = ot.PythonFunction(nbInputs, nbOutputs, mySimulator, gradient=mySimulatorGradient)
```
où `mySimulatorGradient` est une fonction Python qui évalue le gradient.
Calculez à la main des dérivées partielles de la fonction G associée à l'exemple fil rouge. 
Puis définissez la fonction `mySimulatorGradient` qui évalue la matrice Jacobienne. Puisqu'il y a trois variables d'entrée, la liste renvoyée par mySimulatorGradient doit contenir trois éléments. Chaque élément doit contenir une sous-liste de taille 2 contenant les dérivées de chaque sortie. Enfin, construisez la fonction associée avec l'option `gradient`. 

### Exercice 3 : gestion de l'historique d'une fonction Python

**Questions**
* Observer le changement de la valeur de retour de `isHistoryEnabled`
* Quelles sont les méthodes qui permettent de récupérer les historiques des entrées et des sorties ?
* Comment avoir le nombre d’appels à la fonction ?
* Utiliser la méthode `clearHistory` et vérifier que l'historique est vide après cet appel.

### Exercice 4 : fonction symbolique avec 4 entrées

On considère le modèle :
$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 - X_2 X_3 \\
Y_3 &=& 2 X_1 + 3 X_2 + 4 X_4
\end{eqnarray}
$$

**Questions**
* Créer une fonction symbolique pour créer ce nouveau modèle.
* Evaluer la sortie du modèle au point $X=(1,2,3,4)^T$.
* Estimer la moyenne de la sortie par Monte-Carlo simple.

### Exercice 5 : fonction symbolique avec paramètres

On considère le modèle 
$$
\begin{eqnarray}
Y_1 &=& a X_1 + b X_2 \\
Y_2 &=& c X_1 + d X_2
\end{eqnarray}
$$
où a, b, c, d sont des paramètres :
```
a = 12
b = 23
c = -34
d = 45
```

**Questions**
* Créer une fonction symbolique pour créer ce nouveau modèle en utilisant la fonction `str`.
* Evaluer la sortie du modèle au point $X=(1,2,3,4)^T$.

Note : la classe `ParametricFunction` est plus adaptée pour cela.

### Exercice 6 : gradient d'une fonction symbolique

On souhaite vérifier que OT peut calculer la dérivée formelle d’une
fonction symbolique. 

**Questions**
* Définir la fonction `myfunctionSymbolic` comme dans l’exemple fil rouge.
* Créer la variable `myGradient` contenant la dérivée exacte de la fonction. Pour cela, utiliser la méthode `getGradient` de l’objet
`myfunctionSymbolic`. 

* Qu’est-ce qui s’affiche quand on utilise l’instruction suivante ?

`
print(myGradient)
`

* On souhaite évaluer le gradient au point d’entrée suivant :
`
X = (1, 2, 3)
`
Utiliser la méthode `gradient` de l’objet `myGradient` pour évaluer G'(x).

### Exercice 7 : gestion des variables intermédiaires dans une fonction symbolique

On peut définir une fonction symbolique dont l'évaluation est fondée sur des valeurs intermédiaires. Ainsi, la sortie n'est pas seulement une fonction explicite des entrées : on peut définir des résultats intermédiaires et les réutiliser dans une ou plusieurs sorties de la fonction. 

Pour cela, il faut utiliser la séquence d'appel suivante :
```
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
```
où `outputs` est une chaîne de caractères contenant l'expression à évaluer. 

Pour composer cette chaîne de caractère, on peut définir plusieurs expressions, séparées par le caractère ";". De plus, les variables intermédiaires doivent être précédées du mot-clé "`var`". 

Par exemple, dans le cas du modèle dont les entrées sont $X_1$ et $X_2$ et les sorties sont $Y_1$ et $Y_2$ :
$$
\begin{eqnarray}
T &=& X_1 X_2 \\
Y_1 &=& X_1 + T \\
Y_2 &=& X_2 - 3T
\end{eqnarray}
$$
on peut utiliser l'instruction suivante :

In [ ]:
inputs = ['X1', 'X2'] 
outputs = ['Y1', 'Y2']
formula = 'var T := X1*X2; Y1 := X1+T; Y2 := X2-3*T'
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
myFunction([1.,2.])

Définir le modèle dont les entrées sont $X_1$ et $X_2$ et les sorties sont $Y_1$ et $Y_2$ :
$$
\begin{eqnarray}
S &=& X_1 + X_2 \\
T &=& X_1 X_2 \\
Y_1 &=& S + T \\
Y_2 &=& ST
\end{eqnarray}
$$

## Exercice 8 : configurer le nombre de cpus

L'option `n_cpus` de la classe `PythonFunction` permet de configurer le nombre de processeurs. L'implémentation est fondée sur le module `multiprocessing`. Dans cet exercice, on cherche à observer l'effet de cette option sur la performance du calcul.

Pour observer un changement dans la performance nous nous plaçons dans la situation suivante :
- la fonction possède un grand nombre de variables d'entrées,
- la fonction est coûteuse.

Dans ce but, nous définissons la fonction suivante.

In [ ]:
import math

dim = 100

def myHighDimSimulator(x):
    y0=0.
    y1=1.
    for i in range(dim):
        y0=y0+math.exp(x[i])
        y1=y1*math.exp(x[i])
    y=[y0,y1]
    return y

inputHighDimDistribution = ot.ComposedDistribution([ot.Normal()]*dim)
inputHighDimRandomVector = ot.RandomVector(inputHighDimDistribution)

- Utiliser le module `time` pour mesurer la performance de la fonction sans l'option `n_cpus`. Pour observer une durée de simulation significative, augmentez la taille du plan d'expériences ou le nombre de dimensions.
- De même avec l'option `n_cpus`.
- Quelle différence constatez-vous ?